In [1]:
# Use -q for a "quiet" installation to keep the output clean
!pip install -q pandas gspread google-auth-oauthlib google-cloud-aiplatform

In [11]:
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import vertexai
from vertexai.generative_models import GenerativeModel

# --- CONFIGURATION ---
# IMPORTANT: Fill in your Google Cloud Project ID here
GCP_PROJECT_ID = "smartbi-digest"
GCP_LOCATION = "us-east4"

# --- AUTHENTICATION ---
# This will trigger a pop-up to ask for your permission.
print("Authenticating to Google...")
auth.authenticate_user()

# --- INITIALIZE CLIENTS ---
print("Initializing clients for Google Sheets and Vertex AI...")
creds, _ = default()
gc = gspread.authorize(creds)
vertexai.init(project=GCP_PROJECT_ID, location=GCP_LOCATION, credentials=creds)

print("\n✅ Setup Complete! You are now authenticated and ready to proceed.")

Authenticating to Google...
Initializing clients for Google Sheets and Vertex AI...

✅ Setup Complete! You are now authenticated and ready to proceed.


In [12]:
# --- CONFIGURATION ---
# IMPORTANT: Fill in the exact names from your Google Sheets
SPREADSHEET_NAME = "Credit Card Data"  # <-- e.g., "Daily Monetization Report"
WORKSHEET_NAME = "credit_card_data"                         # <-- e.g., "Data Tab" or "Report View"

# --- DATA LOADING & EDA ---
eda_summary = ""
try:
    print(f"🔄 Opening Google Sheet: '{SPREADSHEET_NAME}'...")
    spreadsheet = gc.open(SPREADSHEET_NAME)

    print(f"🔄 Accessing worksheet: '{WORKSHEET_NAME}'...")
    worksheet = spreadsheet.worksheet(WORKSHEET_NAME)

    print("⏳ Loading all data into a DataFrame... (This may take a moment for large sheets)")
    data = worksheet.get_all_records()
    df = pd.DataFrame(data)

    if df.empty:
        print("⚠️ Warning: The worksheet is empty or could not be read properly.")
    else:
        print("✅ Data loaded successfully!")

        print("\n🔬 Performing Exploratory Data Analysis...")
        # 1. Get the shape of the data
        shape = f"Dataset Shape: {df.shape[0]} rows, {df.shape[1]} columns\n"

        # 2. Get the column names
        columns = f"Columns: {', '.join(df.columns)}\n"

        # 3. Get the numerical summary
        # We need to handle cases where there are no numeric columns
        numeric_cols = df.select_dtypes(include='number')
        if not numeric_cols.empty:
            numerical_summary = "Numerical Summary:\n" + numeric_cols.describe().to_string()
        else:
            numerical_summary = "Numerical Summary: No numerical columns found in the data."

        # 4. Consolidate into a single summary string
        eda_summary = shape + columns + "\n" + numerical_summary

        print("\n--- Full Data EDA Summary ---")
        print(eda_summary)
        print("--- End of Summary ---")

except gspread.exceptions.SpreadsheetNotFound:
    print(f"❌ ERROR: Spreadsheet '{SPREADSHEET_NAME}' not found. Please check the name and ensure you have access.")
except gspread.exceptions.WorksheetNotFound:
    print(f"❌ ERROR: Worksheet '{WORKSHEET_NAME}' not found in the spreadsheet. Please check the tab name.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

🔄 Opening Google Sheet: 'Credit Card Data'...
🔄 Accessing worksheet: 'credit_card_data'...
⏳ Loading all data into a DataFrame... (This may take a moment for large sheets)
✅ Data loaded successfully!

🔬 Performing Exploratory Data Analysis...

--- Full Data EDA Summary ---
Dataset Shape: 8950 rows, 18 columns
Columns: CUST_ID, BALANCE, BALANCE_FREQUENCY, PURCHASES, ONEOFF_PURCHASES, INSTALLMENTS_PURCHASES, CASH_ADVANCE, PURCHASES_FREQUENCY, ONEOFF_PURCHASES_FREQUENCY, PURCHASES_INSTALLMENTS_FREQUENCY, CASH_ADVANCE_FREQUENCY, CASH_ADVANCE_TRX, PURCHASES_TRX, CREDIT_LIMIT, PAYMENTS, MINIMUM_PAYMENTS, PRC_FULL_PAYMENT, TENURE

Numerical Summary:
            BALANCE  BALANCE_FREQUENCY     PURCHASES  ONEOFF_PURCHASES  INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY  ONEOFF_PURCHASES_FREQUENCY  PURCHASES_INSTALLMENTS_FREQUENCY  CASH_ADVANCE_FREQUENCY  CASH_ADVANCE_TRX  PURCHASES_TRX      PAYMENTS  PRC_FULL_PAYMENT       TENURE
count   8950.000000        8950.000000   8950.000000   

In [17]:
import json

# --- CALL GEMINI API ---

# Check if the EDA summary was generated successfully before proceeding
if 'eda_summary' in locals() and eda_summary:
    print("🧠 Preparing prompt and calling Gemini API... This may take a few seconds.")

    try:
        # Initialize the specific model we want to use
        # gemini-1.5-flash is fast, multi-modal, has a large context window and is cost-effective.
        model = GenerativeModel("gemini-2.0-flash-lite")

        # This is our powerful "meta-prompt" that instructs the AI
        prompt = f"""
        You are a world-class principal data analyst at a major telecommunications company.
        Your task is to interpret the following Exploratory Data Analysis (EDA) summary of a full dataset and generate
        5 insightful, actionable questions that a business director should investigate next.

        The questions should be sharp, data-driven, and directly inspired by the statistical facts presented.

        --- Full Dataset EDA Summary ---
        {eda_summary}
        --- End of Summary ---

        Based ONLY on this statistical summary, generate a JSON object containing a single key "questions" which holds a list of 5 string questions.
        Example Format: {{"questions": ["question 1", "question 2", ...]}}
        """

        # Call the model
        response = model.generate_content(prompt)

        # --- PARSE AND DISPLAY RESULTS ---
        print("\n🎉 Success! Here are the AI-generated questions:\n")

        # Attempt to parse the JSON response for clean formatting
        try:
            # The model might return the JSON wrapped in markdown, so we clean it
            cleaned_response = response.text.strip().replace("```json", "").replace("```", "")
            data = json.loads(cleaned_response)

            # Print as a numbered list
            for i, question in enumerate(data['questions'], 1):
                print(f"{i}. {question}\n")

        except (json.JSONDecodeError, KeyError) as e:
            print("Could not parse JSON, displaying raw text instead:\n")
            print(response.text)

    except Exception as e:
        print(f"❌ An error occurred while calling the Gemini API: {e}")
else:
    print("⚠️ EDA summary not found. Please run the previous cell successfully before running this one.")

🧠 Preparing prompt and calling Gemini API... This may take a few seconds.

🎉 Success! Here are the AI-generated questions:

1. Given the large standard deviation of 'BALANCE' and 'CASH_ADVANCE', and the high maximum values, what strategies can we implement to mitigate risk associated with high balances and cash advances, and are there any credit limit adjustments needed?

2. The mean 'PURCHASES_FREQUENCY' is only 0.49, and the mean 'ONEOFF_PURCHASES_FREQUENCY' is 0.20, suggesting a substantial portion of customers are not making purchases frequently. What marketing or incentive programs can we develop to encourage higher purchase frequency, particularly for one-off purchases?

3. The mean 'PRC_FULL_PAYMENT' is 0.15, indicating most customers do not pay their balance in full. How does this impact profitability, and what payment plans or features can we offer to encourage customers to pay in full more often?

4. Although the average tenure is 11.5 months with a range from 6 to 12 months,